In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install fuzzywuzzy

In [3]:
import pandas as pd
from fuzzywuzzy import process
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

file_path = '/content/drive/MyDrive/recommendationBeer/cleaned_beer_dataset.csv'
dataframe = pd.read_csv(file_path)

dataframe['ingredients'] = dataframe['ingredients'].apply(lambda x: ', '.join(x.split(', ')) if isinstance(x, str) else '')
dataframe['flavor_profile'] = dataframe['flavor_profile'].apply(lambda x: ', '.join(x.split(', ')) if isinstance(x, str) else '')
dataframe['pairings'] = dataframe['pairings'].apply(lambda x: ', '.join(x.split(', ')) if isinstance(x, str) else '')

dataframe['combined_features'] = (
    dataframe['brand'] + " " +
    dataframe['style'] + " " +
    dataframe['ingredients'] + " " +
    dataframe['flavor_profile'] + " " +
    dataframe['pairings'] + " " +
    dataframe['description']
)

vectorizer = CountVectorizer(stop_words='english', binary=True)
feature_matrix = vectorizer.fit_transform(dataframe['combined_features'])
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)

def get_beer_info(beer_name):
    if beer_name not in dataframe['beer_name'].values:
        suggested_name = process.extractOne(beer_name, dataframe['beer_name'].unique())[0]
        print(f"Did you mean '{suggested_name}'?")
        beer_name = suggested_name

    beer_info = dataframe[dataframe['beer_name'] == beer_name].iloc[0]

    beer_details = {
        'Beer Name': beer_info['beer_name'],
        'Brand': beer_info['brand'],
        'Style': beer_info['style'],
        'ABV': beer_info['abv'],
        'IBU': beer_info['ibu'],
        'Ingredients': beer_info['ingredients'],
        'Flavor Profile': beer_info['flavor_profile'],
        'Pairings': beer_info['pairings'],
        'Description': beer_info['description']
    }

    return beer_details


def get_recommendations(beer_name, cosine_sim=cosine_sim):
    if beer_name not in dataframe['beer_name'].values:
        suggested_name = process.extractOne(beer_name, dataframe['beer_name'].unique())[0]
        print(f"Did you mean '{suggested_name}'?")
        beer_name = suggested_name

    idx = dataframe[dataframe['beer_name'] == beer_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    beer_indices = [i[0] for i in sim_scores if i[0] != idx]
    recommended_beers = dataframe['beer_name'].iloc[beer_indices].drop_duplicates().head(3).tolist()

    print("Recommended Beers:")
    for beer in recommended_beers:
        print(f"- {beer}")

    for beer in recommended_beers:
        print(f"\nDetails for recommended beer '{beer}':")
        beer_details = get_beer_info(beer)
        for key, value in beer_details.items():
            print(f"{key}: {value}")

input_beer = input("Enter the name of the beer: ").strip()
get_recommendations(input_beer)


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Enter the name of the beer: corona
Did you mean 'Corona Extra'?
Recommended Beers:
- Corona Extra
- Bira 91 Blonde Lager
- Cerveza Modelo Especial

Details for recommended beer 'Corona Extra':
Beer Name: Corona Extra
Brand: Grupo Modelo
Style: Lager
ABV: 4.5
IBU: 18
Ingredients: water, barley, hops, corn
Flavor Profile: light, crisp, citrusy, refreshing
Pairings: fish tacos, grilled shrimp, salads
Description: A light and refreshing lager with a crisp, clean finish and subtle citrusy notes, perfect for hot days

Details for recommended beer 'Bira 91 Blonde Lager':
Beer Name: Bira 91 Blonde Lager
Brand: Bira 91
Style: Lager
ABV: 4.9
IBU: 18
Ingredients: water, barley, hops, yeast
Flavor Profile: crisp, light, refreshing, slightly bitter
Pairings: light salads, grilled chicken, fish tacos
Description: Bira 91 Blonde Lager is a crisp and refreshing lager with a slight bitterness, perfect for pairing with light salads, grilled chicken, and fish tacos

Details for recommended beer 'Cerveza 